In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import numpy as np
import os
import albumentations as A
from matplotlib import pyplot as plt

In [5]:
class CancerDataGenerator(tf.keras.utils.Sequence):
    def _init_(self, raw_img_paths, label_img_paths, batch_size, img_size, augment=None):
        self.raw_img_paths = raw_img_paths
        self.label_img_paths = label_img_paths
        self.batch_size = batch_size
        self.img_size = img_size
        self.augment = augment
    
    def _len_(self):
        return int(np.floor(len(self.raw_img_paths)) / self.batch_size)

    def _getitem_(self, index):
        batch_raw_imgs = self.raw_img_paths[index * self.batch_size:(index + 1) * self.batch_size]
        batch_label_imgs = self.label_img_paths[index * self.batch_size:(index + 1) * self.batch_size]

        X, y = self.__data_generation(batch_raw_imgs, batch_label_imgs)
        return X, y
    
    def __data_generation(self, batch_raw_imgs, batch_label_imgs):
        X = np.empty((self.batch_size, self.img_size, self.img_size, 3))
        y = np.empty((self.batch_size, self.img_size, self.img_size, 1))

        for i, (raw_img_path, label_img_path) in enumerate(zip(batch_raw_imgs, batch_label_imgs)):
            raw_img = cv2.imread(raw_img_path)
            raw_img = cv2.resize(raw_img, (self.img_size, self.img_size))

            label_img = cv2.imread(label_img_path, 0)  # Grayscale mask
            label_img = cv2.resize(label_img, (self.img_size, self.img_size))

            if self.augment:
                augmented = self.augment(image=raw_img, mask=label_img)
                raw_img = augmented['image']
                label_img = augmented['mask']

            X[i,] = raw_img / 255.0  # Normalize to 0-1 range
            y[i,] = np.expand_dims(label_img, axis=-1) / 255.0

        return X, y

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import cv2
import matplotlib.pyplot as plt
import albumentations as A

# Define U-Net model
def unet_model(input_size=(256, 256, 3)):
    inputs = layers.Input(input_size)

    # Encoder
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2, 2))(c1)

    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2, 2))(c2)

    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2, 2))(c3)

    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = layers.MaxPooling2D(pool_size=(2, 2))(c4)

    # Bottleneck
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = layers.concatenate([u6, c4])
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = layers.concatenate([u7, c3])
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = layers.concatenate([u8, c2])
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = layers.concatenate([u9, c1], axis=3)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = models.Model(inputs=[inputs], outputs=[outputs])
    return model


In [13]:
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Resize(256, 256),
])


In [33]:
import tensorflow as tf
import os
import numpy as np
import cv2

def load_data_from_directory(image_dir, label_dir, img_size=256, batch_size=8):
    def load_image(image_path):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_image(image, channels=3)
        image = tf.image.resize(image, [img_size, img_size])
        image = image / 255.0  # Normalize to [0, 1]
        return image

    def load_label(label_path):
        label = tf.io.read_file(label_path)
        label = tf.image.decode_image(label, channels=1)
        label = tf.image.resize(label, [img_size, img_size])
        label = label / 255.0  # Normalize to [0, 1]
        return label

    def process_path(image_path, label_path):
        image = load_image(image_path)
        label = load_label(label_path)
        return image, label

    # List all image and label files
    image_files = [os.path.join(image_dir, fname) for fname in os.listdir(image_dir)]
    label_files = [os.path.join(label_dir, fname) for fname in os.listdir(label_dir)]

    # Create a dataset
    dataset = tf.data.Dataset.from_tensor_slices((image_files, label_files))
    dataset = dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Define paths to your data
train_data_path = 'content/data/train/'
validation_data_path = 'content/data/validation/'

# Load datasets
train_dataset = load_data_from_directory(train_data_path, train_data_path)
validation_dataset = load_data_from_directory(validation_data_path, validation_data_path)


ValueError: in user code:

    File "/var/folders/cl/pk1xrw9j75s0g0ghfmbdlx1h0000gn/T/ipykernel_2183/1774263469.py", line 22, in process_path  *
        image = load_image(image_path)
    File "/var/folders/cl/pk1xrw9j75s0g0ghfmbdlx1h0000gn/T/ipykernel_2183/1774263469.py", line 10, in load_image  *
        image = tf.image.resize(image, [img_size, img_size])

    ValueError: 'images' contains no shape.


In [32]:
train_data_path = 'content/data/train/'
validation_data_path = 'content/data/validation/'


In [30]:
train_dataset = load_data_from_directory(train_data_path)
validation_dataset = load_data_from_directory(validation_data_path)


Found 104 files belonging to 2 classes.


ValueError: in user code:

    File "/var/folders/cl/pk1xrw9j75s0g0ghfmbdlx1h0000gn/T/ipykernel_2183/2504432633.py", line 13, in preprocess_images_and_labels  *
        label = tf.image.resize(label, (img_size, img_size))  # Resize label to match image

    ValueError: 'images' must have either 3 or 4 dimensions.


In [24]:
model = unet_model(input_size=(256, 256, 3))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
for image_batch, label_batch in train_dataset.take(1):
    print("Image batch shape:", image_batch.shape)
    print("Label batch shape:", label_batch.shape)


Image batch shape: (8, 256, 256, 3)
Label batch shape: (8, 1)


2024-09-07 00:25:20.160692: W tensorflow/core/lib/png/png_io.cc:89] PNG warning: iCCP: extra compressed data
2024-09-07 00:25:20.208942: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [23]:
model.fit(train_dataset, validation_data=validation_dataset, epochs=25)


Epoch 1/25


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(None, 1), output.shape=(None, 256, 256, 1)

In [ ]:
def predict_image(model, img_path, img_size=256):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (img_size, img_size))
    img = np.expand_dims(img, axis=0) / 255.0  # Normalize

    pred_mask = model.predict(img)[0]
    pred_mask = (pred_mask > 0.5).astype(np.uint8)  # Thresholding the mask

    return pred_mask

# Example usage
pred_mask = predict_image(model, 'path_to_test_image')

plt.imshow(pred_mask.squeeze(), cmap='gray')
plt.show()